In [1]:
import theano
import theano.tensor as tt
import numpy as np
import pandas as pd
import pickle as pkl
import datetime
from collections import OrderedDict
import cProfile
import pstats
%run ../src/sampling_utils.py
%run ../src/shared_utils.py

The always stuff

In [2]:
with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)
    
disease = "covid19"
prediction_region = "germany"

indata = load_daily_data(disease, prediction_region, counties)
data = indata.iloc[:,0:31]
#data = indata.iloc[:,0:31]
rnd_tsel = np.random.Generator(np.random.PCG64(12345))
times_by_day = uniform_times_by_day(data.index, rnd_tsel)

rnd_csel = np.random.Generator(np.random.PCG64(12345))
locations_by_county=uniform_locations_by_county(counties, rnd_csel)

The old stuff

In [3]:
rnd_time = np.random.Generator(np.random.PCG64(12345))
rnd_loc  = np.random.Generator(np.random.PCG64(12345))
rnd_time_pred = np.random.Generator(np.random.PCG64(12345))
rnd_loc_pred  = np.random.Generator(np.random.PCG64(12345))


def gaussian_bf(dx, σ):
    """ spatial basis function """
    σ = np.float32(σ)
    res = tt.zeros_like(dx)
    idx = (abs(dx) < np.float32(5) * σ)  # .nonzero()
    return tt.set_subtensor(res[idx], tt.exp(
        np.float32(-0.5 / (σ**2)) * (dx[idx])**2) / np.float32(np.sqrt(2 * np.pi * σ**2)))


def bspline_bfs(x, knots, P):
    """ temporal basis function
            x: t-delta distance to last knot (horizon 5)
    """
    knots = knots.astype(np.float32)
    idx = ((x >= knots[0]) & (x < knots[-1]))  # .nonzero()
    xx = x[idx]

    N = {}
    for p in range(P + 1):
        for i in range(len(knots) - 1 - p):
            if p == 0:
                N[(i, p)] = tt.where((knots[i] <= xx)
                                     * (xx < knots[i + 1]), 1.0, 0.0)
            else:
                N[(i, p)] = (xx - knots[i]) / (knots[i + p] - knots[i]) * N[(i, p - 1)] + \
                    (knots[i + p + 1] - xx) / (knots[i + p + 1] - knots[i + 1]) * N[(i + 1, p - 1)]

    highest_level = []
    for i in range(len(knots) - 1 - P):
        res = tt.zeros_like(x)
        highest_level.append(tt.set_subtensor(res[idx], N[(i, P)]))
    return highest_level


#NOTE: Do we want basis functions with a longer temporal horizon? // we may want to weight them around fixed days?!
#NOTE: Split this up, so we can get multiple basis functions!
def temporal_bfs(x):
    return bspline_bfs(x, np.array([0, 0, 1, 2, 3, 4, 5]) * 24 * 3600.0, 2) 


def spatial_bfs(x):
    return [gaussian_bf(x, σ) for σ in [6.25, 12.5, 25.0, 50.0]]


def jacobian_sq(latitude, R=6365.902):
    """
        jacobian_sq(latitude)

    Computes the "square root" (Cholesky factor) of the Jacobian of the cartesian projection from polar coordinates (in degrees longitude, latitude) onto cartesian coordinates (in km east/west, north/south) at a given latitude (the projection's Jacobian is invariante wrt. longitude).
    TODO: don't import jacobian_sq from geo_utils to remove potential conflicts
    """
    return R * (np.pi / 180.0) * (abs(tt.cos(tt.deg2rad(latitude))) *
                                  np.array([[1.0, 0.0], [0.0, 0.0]]) + np.array([[0.0, 0.0], [0.0, 1.0]]))


def build_ia_bfs(temporal_bfs, spatial_bfs, profile):
    x1 = tt.fmatrix("x1")
    t1 = tt.fvector("t1")
    # M = tt.fmatrix("M")
    x2 = tt.fmatrix("x2")
    t2 = tt.fvector("t2")

    lat = x1[:, 1].mean()
    M = jacobian_sq(lat)**2

    # (x1,t1) are the to-be-predicted points, (x2,t2) the historic cases

    # spatial distance btw. each points (defined with latitude,longitude) in x1 and x2 with gramian M
    # (a-b)^2 = a^2 + b^2 -2ab; with a,b=vectors
    dx = tt.sqrt(  (x1.dot(M) * x1).sum(axis=1).reshape((-1,  1)) # a^2
                 + (x2.dot(M) * x2).sum(axis=1).reshape(( 1, -1)) # b^2
                 - 2 * x1.dot(M).dot(x2.T) )                      # -2ab

    # temporal distance btw. each times in t1 and t2
    dt = t1.reshape((-1, 1)) - t2.reshape((1, -1))

    ft = tt.stack(temporal_bfs(dt.reshape((-1,))), axis=0) # cast to floats?
    fx = tt.stack(spatial_bfs(dx.reshape((-1,))), axis=0)

    # aggregate contributions of all cases
    contrib = ft.dot(fx.T).reshape((-1,)) / tt.cast(x1.shape[0], "float32")

    return theano.function([t1, x1, t2, x2], contrib, allow_input_downcast=True, profile=profile)

ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs, profile=False)



def sample_ia_effects_old():
    t_data_0 = []
    x_data_0 = []
    t_pred_0 = []
    x_pred_0 = []

    num_tps=5
    d_offs=0 # just to limit the time of test
    c_offs=0 # just to limit the time of test
    days = data.index[d_offs:d_offs+50]
    counties = data.columns[c_offs:c_offs+50]

    _to_timestamp = np.frompyfunc(datetime.datetime.timestamp, 1, 1)
    num_features = len(temporal_bfs(tt.fmatrix("tmp"))) * len(spatial_bfs(tt.fmatrix("tmp")))
    res_0 = np.zeros((len(days), len(counties), num_features), dtype=np.float32)

    for i, day in enumerate(days):
        for j, county in enumerate(counties):
            idx = ((day - pd.Timedelta(days=5)) <= data.index) * (data.index < day)

            t_data, x_data = sample_time_and_space(data.iloc[idx], times_by_day, locations_by_county, rnd_time, rnd_loc)
            t_pred, x_pred = sample_time_and_space(pd.DataFrame(num_tps, index=[day], columns=[county]), times_by_day, locations_by_county, rnd_time_pred, rnd_loc_pred)

            #print("_to_timestamp(t_pred) (types, type1, size, value): ", type(_to_timestamp(t_pred)), type(_to_timestamp(t_pred)[0]), np.shape(_to_timestamp(t_pred)), _to_timestamp(t_pred)[0])
            # => _to_timestamp(t_pred) (types, type1, size, value):  <class 'numpy.ndarray'> <class 'float'> (5,) 1580217693.876309
            #print("x_pred (types, size, value)       : ", type(x_pred), type(x_pred[0]), type(x_pred[0][0]), np.shape(x_pred), x_pred[0][0])        
            # => x_pred (types, size, value)       :  <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.float64'> (5, 2) 10.134378974970323

            res_0[i, j, :] = ia_bfs(_to_timestamp(t_pred), x_pred, _to_timestamp(t_data), x_data)        

            # store all to compare with old algo
            t_data_0 = t_data_0 + t_data.tolist()
            x_data_0 = x_data_0 + x_data.tolist()
            t_pred_0 = t_pred_0 + t_pred.tolist()
            x_pred_0 = x_pred_0 + x_pred.tolist()
            
    return t_data_0, x_data_0, t_pred_0, x_pred_0

t_data_0, x_data_0, t_pred_0, x_pred_0 = sample_ia_effects_old()


TypeError: sample_time_and_space() missing 6 required positional arguments: 'day_of_smpl', 'av_times_per_smpl', 'cnty_of_smpl', 'av_locs_per_smpl', 'rnd_time', and 'rnd_loc'

The new stuff

In [ ]:

#ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs, profile=False)
#%%timeit
#cp = cProfile.Profile()
#cp.enable()
samp = iaeffect_sampler(data, times_by_day, locations_by_county, temporal_bfs, spatial_bfs)
#cp.disable()
#cp.create_stats()
#p = pstats.Stats()
#p.strip_dirs()
#cp.dump_stats('profilenumbasingle.dmp')

In [ ]:
#print(np.shape(data))
#print(np.shape(data.index))
#print(np.shape(data.columns))
#print(type(samp), type(samp[0]), type(samp[0][0]), type(samp[0][0][0]))
#print(not 5)